In [16]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-13b-instruct-full-jaster-v1.0")
model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-13b-instruct-full-jaster-v1.0", device_map="auto",
                                              torch_dtype=torch.float16)


Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.68s/it]


In [23]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              max_new_tokens=1,
              )

In [24]:
instruction="日本語の文ペアの意味がどのくらい近いかを判定し、類似度を0.0〜5.0までの間の値で付与してください。0.0に近いほど文ペアの意味が異なり、5.0に近いほど文ペアの意味が似ていることを表しています。整数値のみを返し、それ以外には何も含めないことを厳守してください。"
t1=""
t2=""
problem="文1：レンガの建物の前を、乳母車を押した女性が歩いています。\n文2：厩舎で馬と女性とが寄り添っています。"
problem_template="文1：{t1}\n文2：{t2}"

In [32]:
txt="指示:" +instruction+"\n入力: "+ problem+"\n出力: "


In [48]:
import pandas as pd
df=pd.read_csv("../datasets/JSNLI_train.tsv",sep="\t",
              )
df.columns=["label","sentence1","sentence2"]
df["sentence1"]=df["sentence1"].str.replace(" ","")
df["sentence2"]=df["sentence2"].str.replace(" ","")

#shuffle dataframe
df=df.sample(frac=1)
df

,label,sentence1,sentence2
217931,entailment,セメントミキサーの横にある外に立っているブロンドの髪と白人男性。,白人は外にいます。
287348,neutral,リムジンの前の歩道で２人がキスをしている間、人々は通り過ぎます。,二人は付き合っています
399952,neutral,大観衆の前でゲームをするサッカー選手。,サッカー選手は、売り切れの観客の前で世界選手権でプレーします。
56218,neutral,葉っぱの巨大な山からカメラを覗く二人の少女。,女の子はカメラに微笑んでいます。
48702,contradiction,ひげを生やした男が動物園でクマと寄り添います。,ひげを生やした男がボートに横たわっています。
...,...,...,...
118647,entailment,懐中電灯を頭につけた男は、針金のついたローリングケースに手を入れています。,男は暗い場所にいます。
123458,neutral,カーニバルは、青いスウェットパンツを着て、チョークで塗られた黒いタンクトップをテントの外に立...,労働者は彼のシフト後に休んでいます
488129,contradiction,若い女の子が本から絵をコピーして絵を描く練習をしています。,古い僧ｅｓｔは、本から絵をコピーして絵を描きます。
10364,entailment,暗褐色の犬が急速に揺れています。,犬が揺れます。


In [49]:
loader=iter(df.values)
next(loader)

array(['entailment', 'セメントミキサーの横にある外に立っているブロンドの髪と白人男性。', '白人は外にいます。'],
      dtype=object)

In [50]:


def eval(pipe,txt):
    return pipe(txt)[0]["generated_text"][len(txt):]

instruction="""以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
###指示 日本語の文ペアの意味がどのくらい近いかを判定し、類似度を0.0〜5.0までの間の値で付与してください。0.0に近いほど文ペアの意味が異なり、5.0に近いほど文ペアの意味が似ていることを表しています。整数値のみを返し、それ以外には何も含めないことを厳守してください。"""
def make_problem(t1,t2):
    problem=f"文1：{t1}\n文2：{t2}"
    txt=instruction+"\n"+ problem+"\n\n###応答: \n"
    return txt

In [52]:
n_problems=100
problem_list=[]
for i in range(n_problems):
    d=next(loader)
    s1=d[1]
    s2=d[2]

    text=make_problem(s1,s2)
    score=eval(pipe,text)

    print(score,s1,s2)

Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


2 岩の山の近くの芝生の上に立っている白い犬と空気中のジャンプ黒犬。 黒犬はフリスビーを捕まえています。
1 カラフルな服を着た女の子が、ベッドの上でカラーブロックのキルトでジャンプしています。 女の子は屋内にいます。
0 男が雪の丘を滑り降ります。 男はビーチに横たわっています。
2 オレンジ色のバスケットボール選手がボールを持ち、青と白の３人が防御します。 人々はバスケットボールをしています。
2 中規模の部屋で人々のグループを講義する男。 人々は講義を聞くことを楽しんでいます。
1 帽子をかぶった年配の男性が右を向くテントの下に座っています。 男はキャンプしています


Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


0 果物、野菜、花のあるカラフルな屋外市場の人々。 人々は屋内ショッピングモールにいます。
1 マスコミからインタビューを受けているアラブの女性。 インタビューされた背の高い人間
2 路地でサッカーをしている子供。 子供は彼がチームを作ることができるように彼のサッカーのスキルを練習しています
0 自転車に乗った男が通りを下っていき、男が歩道の上に縞模様のシャツを着て立っていて、その上にオレンジ色の円錐形の看板の写真を撮っている。 二人とも道をスケートボードに乗っています。
2 数人がレストランで屋外で食事をしています。 誕生日パーティーのために外で食べる人々のグループ


Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


...

2 新聞を読む赤いフランネルを着た男。 男はいつも小説を読むために彼の赤いフランネルを着ています。
3 雨の中で家に歩いている２人の男の子。 雨の中で二人の少年がキャッチをしている。
1 素敵なズボン、ベルト、ネクタイを持つ少年は階段から飛び降りています 汚れた服を着た年上の少年が階段で遊んだ。
3 ３人が大きな機械を通りに押しています。 数人の人が外で機械を動かしています。
0 カキパンツを着た上半身裸の少年が沼にひざまずいており、誰かが近くで水しぶきを上げています。 フェアでは観覧車に２人が乗っています。
0 乱雑な家のフィットネスマシン上の女性。 女性がうまくいく


Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


2 赤いシャツを着た男がトレイルバイクを修理するのを助ける青いシャツを着た男 赤いシャツを着た男が車を修理するのを助ける青いシャツを着た男。
2 デスクで仕事をしている男性が眠りに落ちる。 従業員が眠ろうとしています
0 ロープからぶら下がっている壊れやすいボールを観察する青いシャツを着た少年。 誰もシャツを持っていません。
4 誰かが犬を歩いています。 人が犬を歩いています。
3 茶色の犬が黒と白の犬に飛びついています。 大きな犬が黒と白の犬に飛びついています。
2 ２人の女性が水の横に立ち、使い捨てカメラで写真を撮ります。 女性は水の近くにいます。


Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:7 for open-end generation.


1 長髪の男がビーチの写真を撮っている。 カメラを持った男。
0 赤い花を保持しているウェディングドレスの女性。 女性が作業服を着てケンタッキー州にリンゴの木を植えています。
0 群衆は男がサーフィンするのを見ています。 群衆は泣いています。
2 中東の市場では自家製の食品を販売しています。 市場は食料を販売しています。
1 がんサバイバーは治療のために競います。 がんサバイバーが昼寝をしている
1 帆をあげる５人 帆を上げる太った人間
1 淡い日焼けした犬は、口の外側に棒を持っています。 紫の犬が草の中に座っています。


In [35]:
score

'1'